In [47]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stopWords = set(word.strip().lower() for word in open("stop_words_en.txt"))

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
                                                      
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stopWords]
                                                     
    for word in words:                                                      
        print "%s\t%s\t%d" % (word, article_id, 1)

Overwriting mapper.py


In [48]:
%%writefile reducer.py

import sys

current_word = None
current_article = None
article_count = 0

for line in sys.stdin:
    try:
        word, article, count = line.strip().split('\t', 2)
    except ValueError as e:
        continue
        
    if current_word != word:
        if current_word:
            print "%s\t%d\t%d" % (current_word, 0, article_count)
        current_word = word
        current_article = article
        article_count = 1
    elif current_article != article:        
        article_count +=1        
        current_article = article

if current_word:
    print "%s\t%d\t%d" % (current_word, 0, article_count)

Overwriting reducer.py


In [49]:
%%bash

ARTICLE_COUNT=article_count
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${ARTICLE_COUNT} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming word count per article" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D mapreduce.partition.keycomparator.options="-k1,2" \
    -D stream.num.map.output.key.fields=2 \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${ARTICLE_COUNT} > /dev/null

hdfs dfs -cat ${ARTICLE_COUNT}/part-00000 | head

𐡎	0	1
𐡆	0	1
𐑋	0	1
𐑃	0	1
𐐺	0	1
𐐲	0	1
𐐪	0	1
ﻫ	0	1
ﻣ	0	1
ﺻـ	0	1


17/12/01 13:31:13 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/01 13:31:13 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/01 13:31:16 INFO mapred.FileInputFormat: Total input files to process : 1
17/12/01 13:31:16 INFO mapreduce.JobSubmitter: number of splits:2
17/12/01 13:31:17 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1512129487187_0013
17/12/01 13:31:17 INFO impl.YarnClientImpl: Submitted application application_1512129487187_0013
17/12/01 13:31:17 INFO mapreduce.Job: The url to track the job: http://bb1396e94d95:8088/proxy/application_1512129487187_0013/
17/12/01 13:31:17 INFO mapreduce.Job: Running job: job_1512129487187_0013
17/12/01 13:31:23 INFO mapreduce.Job: Job job_1512129487187_0013 running in uber mode : false
17/12/01 13:31:23 INFO mapreduce.Job:  map 0% reduce 0%
17/12/01 13:31:39 INFO mapreduce.Job:  map 65% reduce 0%
17/12/01 13:31:51 INFO mapreduce.Job:  map 100% reduce 0%
17/12/01 13:32:00 INFO

In [50]:
!hdfs dfs -cat article_count/part-00000 | head -10


𐡎	0	1
𐡆	0	1
𐑋	0	1
𐑃	0	1
𐐺	0	1
𐐲	0	1
𐐪	0	1
ﻫ	0	1
ﻣ	0	1
ﺻـ	0	1
cat: Unable to write to output stream.


In [67]:
%%writefile mapper1.py

import sys
import re
import os

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stopWords = set(word.strip().lower() for word in open("stop_words_en.txt"))

for line in sys.stdin:
   
    if "article_count" in os.environ["mapreduce_map_input_file"]:
        print line.strip()
    else:
        try:
            article_id, text = unicode(line.strip()).split('\t', 1)
        except ValueError as e:
            continue
                                                      
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        words = [word.lower() for word in words]
        words = [word for word in words if word not in stopWords]
                                                     
        article_size = len(words)
        
        for word in words:               
            print "%s\t%s\t%d" % (word, article_id, article_size)
    


Overwriting mapper1.py


In [95]:
%%writefile reducer1.py

from __future__ import print_function
from math import log

import sys

current_word = None
current_article = None
current_article_size = 0
articles_count = 0
word_count = 0

def printResult():
    tf = 1.0*word_count/current_article_size
    idf = 1.0/log(1+articles_count)
    print(current_word, current_article, tf*idf, sep="\t")


for line in sys.stdin:
    try:
        word, article, count = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue
    
    if article=="0":
        if current_article:
            printResult()
        word_count = 0
        current_article = None
        current_article_size = 0        
        articles_count = count
        current_word = word
    elif current_article != article:
            if current_article:
                printResult()
            current_article = article
            current_article_size = count
            word_count = 0

if current_article:
    printResult()

Overwriting reducer1.py


In [96]:
%%bash

OUT=result
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming tf-idf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D mapreduce.partition.keycomparator.options="-k1,2" \
    -D stream.num.map.output.key.fields=2 \
    -files mapper1.py,reducer1.py,/datasets/stop_words_en.txt \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input article_count,/data/wiki/en_articles_part \
    -output ${OUT} > /dev/null

hdfs dfs -cat ${OUT}/part-00000 | head

	1878	398
	
	3270	355
	
	6556	578
	
	8778	2425
	
	8833	1070
	


17/12/01 15:01:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/01 15:01:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/01 15:01:13 INFO mapred.FileInputFormat: Total input files to process : 9
17/12/01 15:01:13 INFO mapreduce.JobSubmitter: number of splits:10
17/12/01 15:01:14 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1512129487187_0035
17/12/01 15:01:14 INFO impl.YarnClientImpl: Submitted application application_1512129487187_0035
17/12/01 15:01:14 INFO mapreduce.Job: The url to track the job: http://bb1396e94d95:8088/proxy/application_1512129487187_0035/
17/12/01 15:01:14 INFO mapreduce.Job: Running job: job_1512129487187_0035
17/12/01 15:01:19 INFO mapreduce.Job: Job job_1512129487187_0035 running in uber mode : false
17/12/01 15:01:19 INFO mapreduce.Job:  map 0% reduce 0%
17/12/01 15:01:29 INFO mapreduce.Job:  map 20% reduce 0%
17/12/01 15:01:30 INFO mapreduce.Job:  map 40% reduce 0%
17/12/01 15:01:37 INFO